In [1]:
# Imports
import numpy as np
import spikeinterface as si

In [2]:
# Properties of the in-memory dataset
num_channels=7
samplerate=30000
duration=20
num_timepoints=int(samplerate*duration)
num_units=4
num_events=1000

In [3]:
# Generate a pure-noise timeseries dataset and a linear geometry
timeseries=np.random.normal(0,10,(num_channels,num_timepoints))
geom=np.zeros((num_channels,2))
geom[:,0]=range(num_channels)

# Define the in-memory recording extractor
RX=si.NumpyRecordingExtractor(timeseries=timeseries,geom=geom,samplerate=samplerate)

# Generate some random events
times=np.int_(np.sort(np.random.uniform(0,num_timepoints,num_events)))
labels=np.random.randint(1,num_units+1,size=num_events)
    
# Define the in-memory sorting extractor
SX=si.NumpySortingExtractor()
for k in range(1,num_units+1):
    times_k=times[np.where(labels==k)[0]]
    SX.addUnit(unit_id=k,times=times_k)

In [4]:
# Demonstrate the API for extracting information
print('Unit ids = {}'.format(SX.getUnitIds()))
st=SX.getUnitSpikeTrain(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=SX.getUnitSpikeTrain(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1, 2, 3, 4]
Num. events for unit 1 = 262
Num. events for first second of unit 1 = 8


In [5]:
# Write the input/output in the MountainSort format
si.MdaRecordingExtractor.writeRecording(recording=RX,save_path='sample_mountainsort_dataset')
si.MdaSortingExtractor.writeSorting(sorting=SX,save_path='sample_mountainsort_dataset/firings_true.mda')

In [6]:
# Read this dataset with the Mda input extractor
RX2=si.MdaRecordingExtractor(dataset_directory='sample_mountainsort_dataset')
SX2=si.MdaSortingExtractor(firings_file='sample_mountainsort_dataset/firings_true.mda')

In [7]:
# We should get he same information as above
print('Unit ids = {}'.format(SX2.getUnitIds()))
st=SX2.getUnitSpikeTrain(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=SX2.getUnitSpikeTrain(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1 2 3 4]
Num. events for unit 1 = 262
Num. events for first second of unit 1 = 8


In [8]:
#Calculate a unit property and store it in my SortingExtractor
full_spike_train = SX2.getUnitSpikeTrain(unit_id=1)
firing_rate = float(len(full_spike_train))/RX2.getNumFrames()
SX2.setUnitProperty(unit_id=1, property_name='firing_rate', value=firing_rate)
print('Average firing rate during the recording of unit 1 = {}'.format(SX2.getUnitProperty(unit_id=1, property_name='firing_rate')))

Average firing rate during the recording of unit 1 = 0.0004366666666666667


In [16]:
#Can get a the sub-dataset from the sorting
SX3=si.SubSortingExtractor(parent_sorting=SX2,unit_ids=[1, 2],
                           start_frame=10000,end_frame=20000)

In [17]:
print('Num. units = {}'.format(len(SX3.getUnitIds())))
print('Average firing rate of units1 during frames 14000-16000 = {}'.format(
                        float(len(SX3.getUnitSpikeTrain(unit_id=1)))/6000))

Num. units = 2
Average firing rate of units1 during frames 14000-16000 = 0.0006666666666666666
